In [1]:
#라이브러리 불러오기
import pandas as pd
import numpy as np
from konlpy.tag import Twitter
from konlpy.utils import pprint
from gensim import models

In [2]:
#크롤링 결과를 포스팅 단위로 분리 
doc =open('삼성역.csv')
data = doc.read()
doc.close()
phrase = data.split('\n')
#문서의 이름을 name으로 지정 ex)삼성역-->name= 삼성역
name=doc.name

In [3]:
#포스팅에서 한글 외 문자 제거
import re
hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')


In [4]:
#형태소 문장 단위 추출
dic_Twitter = []
for i in range(len(phrase)):
    p = hangul.sub(' ', phrase[i])
    s = Twitter().pos(p, stem=True, norm=True)  ##norm, stem 옵션 추가
    dic_Twitter.append(s)
print(dic_Twitter[:2])

[[('구글', 'Noun'), ('캠퍼스', 'Noun'), ('캠퍼스', 'Noun'), ('서울', 'Noun'), ('삼성역', 'Noun'), ('넘다', 'Verb'), ('들어오다', 'Verb'), ('싶다', 'Verb'), ('아늑하구', 'Noun'), ('좋다', 'Adjective'), ('회사', 'Noun'), ('지하', 'Noun'), ('층', 'Noun'), ('라운지', 'Noun'), ('직딩', 'Noun'), ('직딩', 'Noun'), ('스타', 'Noun'), ('그램', 'Noun'), ('회사', 'Noun'), ('유니콘', 'Noun'), ('웰컴', 'Noun'), ('팩', 'Noun'), ('일상', 'Noun'), ('일기', 'Noun'), ('데', 'Noun'), ('일리', 'Noun'), ('맞팔', 'Noun'), ('맞팔', 'Noun'), ('해', 'Josa'), ('요', 'Noun'), ('맞팔', 'Noun'), ('스타', 'Noun'), ('그램', 'Noun'), ('선팔', 'Noun'), ('하다', 'Verb'), ('맞팔', 'Noun')], [('멋지다', 'Adjective'), ('흘르다', 'Verb'), ('넘치다', 'Verb'), ('블랙', 'Noun'), ('프렌치', 'Noun'), ('불독', 'Noun'), ('으악', 'Adverb'), ('보기', 'Noun'), ('만', 'Josa'), ('해도', 'Noun'), ('안다', 'Verb'), ('싶다', 'Verb'), ('블랙', 'Noun'), ('왕자', 'Noun'), ('님', 'Suffix'), ('을', 'Josa'), ('소개', 'Noun'), ('하다', 'Verb'), ('찐빵', 'Noun'), ('같이', 'Josa'), ('통통', 'Noun'), ('한', 'Josa'), ('두상', 'Noun'), ('은', 'Josa'), ('물론', 'Noun'), ('건

In [5]:
#형태소 분석 결과에서 Adjective와 noun품사 결과만 추출하고 특정 단어들(스타, 그램, 역이름, 스럽다, 저렇다 등등)을 제거
#(품사,단어) 튜블에서 단어만 추출
dic_ad_noun=[]
for twit in dic_Twitter:
    c = [x[0] for x in twit if x[1] =="Adjective" or x[1] == "Noun" and len(x[0]) >1 and x[0] not in ["스타","그램",name[:-4],name[:-5],"이다","있다","없다","같다","아니다","그렇다","이렇다","많다","어떻다",
                                             "스럽다","저렇다","인스타그램","맞팔","선팔","이벤트","응모"]]
    dic_ad_noun.append(c)
print(dic_ad_noun[:2])

[['구글', '캠퍼스', '캠퍼스', '서울', '아늑하구', '좋다', '회사', '지하', '라운지', '직딩', '직딩', '회사', '유니콘', '웰컴', '일상', '일기', '일리'], ['멋지다', '블랙', '프렌치', '불독', '보기', '해도', '블랙', '왕자', '소개', '찐빵', '통통', '두상', '물론', '건강미', '몸매', '블랙', '왕자', '모습', '얼마나', '이쁘다', '멋지다', '가득', '블랙', '아가', '궁금하다', '지금', '바로', '문의', '다음', '카페', '퍼피', '클럽', '검색', '대한민국', '대표', '프리미엄', '강아지', '분양', '퍼피', '클럽', '분양', '문의', '프렌치', '불독', '프렌치', '불독', '프렌치', '불독', '분양', '불독', '분양', '불독', '프렌치', '불독', '사랑', '이다', '크림', '프렌치', '불독', '홍대', '가로수길', '압구정', '신촌', '이대', '이태원', '여의도', '광화문', '신촌', '인사동', '역삼', '노원', '청담동', '논현', '혜화역', '반포', '판교', '송도', '영등포', '일산']]


In [7]:
#word2vec 모델을 만든다, 차원에 단어를 배치한다
w2v = models.Word2Vec(dic_ad_noun)

In [8]:
#'맛있다'인근 단어를 검색한다
w2v.similar_by_word("맛있다")

[('또띠아', 0.8967073559761047),
 ('꿀맛', 0.8960040807723999),
 ('청포도', 0.8938004374504089),
 ('연어', 0.8929448127746582),
 ('맵다', 0.8836594223976135),
 ('음식', 0.882120668888092),
 ('치돈', 0.8801458477973938),
 ('짬뽕', 0.879653811454773),
 ('햄버거', 0.877011775970459),
 ('샐러드', 0.8726705312728882)]